# Resolution parameter on Sierpinski

In [1]:
import colorcet as cc
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [2]:
%run -i functions.ipynb

In [3]:
def plot_leiden_communities_on_axes(graph, layout, resolution, ax, title_suffix=""):
    """
    Clusters a given graph using Leiden with a specified resolution and plots it
    onto the provided Matplotlib Axes, coloring vertices by community.
    """
    ax.clear()
    
    if graph.vcount() == 0:
        ax.set_title(f"Sierpiński Triangle - No vertices")
        ax.axis('off')
        return

    communities = graph.community_leiden(objective_function="modularity", resolution=resolution)
    
    num_communities = len(communities)
    palette = cc.glasbey_dark
    vertex_colors = [palette[membership_id % len(palette)] for membership_id in communities.membership]
    
    ig.plot(
        graph,
        target=ax,
        layout=layout,
        vertex_size=32,
        vertex_color=vertex_colors,
        vertex_label=None,
        edge_color="black",
        edge_width=1.5,
        bbox=(0, 0, 600, 600),
        margin=20,
    )
    
    ax.set_title(f"Resolution: {resolution:.3f} ({num_communities} comms) {title_suffix}")
    ax.set_aspect('equal', adjustable='box')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')

def create_interactive_resolution_param_tabs():
    """
    Generates and displays an interactive tabbed interface for visualizing Leiden clustering
    on a Sierpiński graph at different resolution parameters.
    """
    SIERPINSKI_DEPTH = 4
    sierpinski_graph, sierpinski_layout = get_sierpinski_graph_and_layout(SIERPINSKI_DEPTH)
    
    print(f"Generated Sierpiński Graph (Depth {SIERPINSKI_DEPTH}):")
    print(f"  Vertices: {sierpinski_graph.vcount()}")
    print(f"  Edges: {sierpinski_graph.ecount()}")
    
    resolution_values = [10 ** i for i in np.arange(-1.5, 1.51, 0.75)]
    output_widgets = [widgets.Output() for _ in resolution_values]
    
    tab_titles = []
    for i, res in enumerate(resolution_values):
        with output_widgets[i]:
            fig, ax = plt.subplots(figsize=(7, 7))
            
            plot_leiden_communities_on_axes(
                sierpinski_graph,
                sierpinski_layout,
                resolution=res,
                ax=ax,
                title_suffix=f"(Depth {SIERPINSKI_DEPTH})"
            )
            plt.tight_layout()
            plt.show()
        
        tab_titles.append(f"Res: {res:.3f}")
    
    leiden_tabs = widgets.Tab()
    leiden_tabs.children = output_widgets
    
    for i, title in enumerate(tab_titles):
        leiden_tabs.set_title(i, title)
    
    print("\nSierpiński Graph with Leiden Communities (varying resolution):")
    display(leiden_tabs)

In [4]:
create_interactive_resolution_param_tabs()

Generated Sierpiński Graph (Depth 4):
  Vertices: 123
  Edges: 243

Sierpiński Graph with Leiden Communities (varying resolution):


The above images illustrate how the *resolution parameter* in the Leiden community detection algorithm affects the number and size of communities found in a *Sierpinski graph*.

---

#### Key observations

**Low Resolution (Res: 0.032 and 0.178):** A low value for the resolution parameter encourages the algorithm to find a small number of large communities. In this case, it identifies just 1 or 3 communities, corresponding to the three main triangular segments of the Sierpinski graph. This is a coarse-grained view of the network's structure.

**Default Resolution (Res: 1.000):** The default value of 1.000 provides a balanced result. It identifies 9 communities, which are smaller than those found at low resolution, revealing more of the network's internal structure without being overly fragmented.

**High Resolution (Res: 5.623 and 31.623):** A high resolution value makes the algorithm more sensitive to small, tightly-knit groups. As the value increases, the number of communities grows significantly.

---

#### Conclusion

The resolution parameter acts as a control knob for the granularity of community detection. A lower value yields fewer, larger communities, while a higher value results in more numerous, smaller communities. This demonstrates the hierarchical nature of the Sierpinski graph, where communities can be detected at different scales depending on the chosen resolution.